In [824]:
import numpy as np
import tensorflow as tf

In [870]:
# N observations with D parameters
N = 2**13
D = 500

In [871]:
beta_true = np.random.uniform(-20, 20, D).reshape(D, 1)

In [872]:
def generateData(n, beta, link='linear', sigma=3.0):
    """Generate data for GLM.
    
    # Arguments
        N: number of observations
        beta: regression coefficient vector
        link: linear or logistic
        sigma: standard deviation of Gaussian noise (only for linear regression)
        
    # Returns
        A list of design matrix and response vector
    """
    d = beta.shape[0]
    X = np.random.normal(0, 1, n * d).reshape(n, d)
    eta = np.dot(X, beta)
    if link == 'linear':
        mu = eta
        Y = eta + np.random.normal(0, sigma, n).reshape(n, 1)
    elif link == 'logistic':
        mu = 1.0 / np.exp(-eta)
        Y = (mu > 0.5).astype(float)
    return(X, Y)

In [873]:
dataX, dataY = generateData(N, beta_true, link='linear')

In [874]:
np.savetxt('dataX.csv', dataX)
np.savetxt('dataY.csv', dataY)

In [875]:
# best linear unbiased estimate
def lm(X, Y):
    estimate = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.transpose(X)), Y)
    return(estimate)

In [876]:
beta_LS = lm(dataX, dataY)

In [877]:
# placeholding tensors and variable
X = tf.placeholder('float', [None, D]) 
Y = tf.placeholder('float', [None, 1])
beta = tf.Variable(tf.random_normal([D, 1], stddev=1.0))

In [878]:
# linear regression with mean squared error
Y_hat = tf.matmul(X, beta)
loss = tf.reduce_sum(tf.square(Y - tf.matmul(X, beta)))

In [879]:
# logistic regression with log loss
eta = tf.matmul(X, beta)
p = 1.0 / tf.exp(-eta)
#loss = -1.0 * (tf.reduce_sum(Y * tf.log(p) + (1.0 - Y) * tf.log(1 - p)))

In [880]:
# gradient of loss function
grad = tf.gradients(loss, beta)

In [881]:
# hessian (or use tf.hessians in Tensorflow 1.0)
def compute_hessian():
    for i in range(D):
        # element in the gradient vector
        dfdx_i = tf.slice(grad[0], begin=[i, 0], size=[1, 1])
        # differentiate again
        ddfdx2_i = tf.gradients(dfdx_i, beta)[0]
        # combine second derivative vectors
        if i == 0:
            hess = ddfdx2_i
        else:
            hess = tf.concat([hess, ddfdx2_i], 1)
    return(hess)

hessian = compute_hessian()

In [882]:
# fisher information
fisher = tf.matrix_inverse(hessian)

In [883]:
# update beta by delta
delta = tf.placeholder('float', [D, 1])
descent = beta.assign_add(delta)

In [884]:
def hadamard(k):
    """Create standard Hadamard matrix.
    
    # Arguments
        k: power of 2
    
    # Returns
        A Hadamard matrix of size 2 ^ k
    """
    H2 = np.ones((2, 2))
    H2[1, 1] = -1.0
    H2 = H2 / np.sqrt(2)
    H = 1.0
    for i in range(0, k):
        H = np.kron(H2, H)
    return(H)

In [885]:
def sketch(X, Y, gamma=4, method='gaussian', sampling=False, bootstrap=False):
    """Randomly data projection.
    
    # Arguments
        X: design matrix
        Y: response vector
        D: number of parameters
        (R: projection dimension = gamma * D)
        
    # Usage
        Subsampling: method='none', sampling=True, bootstrap=True
        i.i.d Gaussian: method='gaussian', sampling=False, bootstrap=False
        Hadamard: method='hadamard', sampling=True, bootstrap=False
    
    # Returns
        A list of design matrix and response vector in projection space
    """
    N = X.shape[0]
    D = X.shape[1]
    R = gamma*D
    SX = X
    SY = Y
    if method == 'gaussian':
        S = np.random.normal(0, 1, R * N).reshape(R, N)
        SX = np.dot(S, X)
        SY = np.dot(S, Y)
    elif method == 'hadamard':
        H = hadamard(int(np.log2(N)))
        temp = np.ones(N)
        temp[np.random.randint(low=0, high=N, size=int(N / 2))] = -1.0
        D = np.diag(temp)
        S = np.dot(H, D)
        SX = np.dot(S, X)
        SY = np.dot(S, Y)
    if sampling:
        select = np.random.choice(np.array(range(0, N)), size=R, replace=bootstrap)
        SX = SX[select, :]
        SY = SY[select, :]
    return(SX, SY)

In [845]:
# sample path
MAXITER = 100
path = np.zeros((MAXITER, D))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(MAXITER):
        # sketching
        sketchX, sketchY = sketch(dataX, dataY, gamma=4, method='gaussian', sampling=False, bootstrap=False)
        # compute gradient
        g = sess.run(grad, feed_dict={X: sketchX, Y: sketchY})[0]
        # compute hessian
        I = sess.run(fisher, feed_dict={X: sketchX, Y: sketchY})
        # update beta
        sess.run(descent, feed_dict={delta : -1/(i + 1)*np.dot(I, g)})
        path[i, :] = np.transpose(beta.eval())

In [886]:
# sample path with half-stepping
# TODO: 
#     Improve stopping criteria
#     Implement Line Search for descent distance
MAXITER = 30
TOL = 10**(-6)
path = np.zeros((MAXITER, D))
step = 1
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(MAXITER):
        # sketching
        sketchX, sketchY = sketch(dataX, dataY, gamma=4, method='gaussian', sampling=False, bootstrap=False)
        # compute gradient
        g = sess.run(grad, feed_dict={X: sketchX, Y: sketchY})[0]
        # compute hessian
        I = sess.run(fisher, feed_dict={X: sketchX, Y: sketchY})
        # update beta
        sess.run(descent, feed_dict={delta : -step*np.dot(I, g)})
        if i == 0:
            current_loss = sess.run(loss, feed_dict={X: dataX, Y: dataY, beta: beta.eval()})
        else:
            new_loss = sess.run(loss, feed_dict={X: dataX, Y: dataY, beta: beta.eval()})
            while new_loss > current_loss:
                new_loss = sess.run(loss, feed_dict={X: dataX, Y: dataY, beta: beta.eval()})
                step = step/2
                sess.run(descent, feed_dict={delta: step*np.dot(I, g)})
                if step < TOL or abs(current_loss - new_loss) < TOL:
                    step = 2*step
                    break
            current_loss = new_loss
        print(current_loss)
        path[i, :] = np.transpose(beta.eval())

98412.9
94198.4
82477.3
80182.9
77717.6
75872.2
75128.6
74775.5
74393.5
74280.9
74180.8
73913.9
73513.6
73337.4
73118.9
72899.3
72704.3
72476.3
72358.2
72263.8
72157.7
72059.5
71893.7
71875.3
71816.9
71758.9
71744.7
71627.2
71606.5
71493.4


In [893]:
#np.savetxt('gaussian.csv', path)
beta_sketch = path[i, :]
ratio = np.mean(np.square(beta_sketch - np.transpose(beta_LS))) / np.linalg.norm(beta_LS)
print(ratio)
np.savetxt('gaussian.csv', path)

9.24940104495e-07


In [808]:
# arrays to store results
B_hat = np.zeros((100, D))
B_sketch = np.zeros((100, D))

# simulate 100 times
for i in range(0, 100):
    dataX, dataY = generateData(N, beta_true, link='linear')
    B_hat[i, :] = np.transpose(lm(dataX, dataY))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for j in range(0, 10):
            sketchX, sketchY = sketch(dataX, dataY, 512, method='hadamard', sampling=True, bootstrap=False)
            g = sess.run(grad, feed_dict={X: sketchX, Y: sketchY})[0]
            I = sess.run(fisher, feed_dict={X: sketchX, Y: sketchY})
            sess.run(descent, feed_dict={delta : -np.dot(I, g)})
        B_sketch[i, :] = np.transpose(beta.eval())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.glo

In [809]:
ratio = np.mean(np.square(B_sketch - np.transpose(beta_true))) / np.mean(np.square(B_hat - np.transpose(beta_true)))
print(ratio)

2.10456919353
